In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.feature_selection import RFE, RFECV
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [9]:
df = pd.read_csv('/kaggle/input/trexquant-2025-summer-intern-challenge-iitkgp/TQ_Train_Kaggle.csv')
df

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,target
0,1,1.65,-3.48,NaN,0.12,0.22,0.20,NaN,-0.29,-0.36,-1.88,-2.82,NaN,NaN,NaN,0.09,-0.72,0.12,1.11,3.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.41,-0.41,0.44,0.00,NaN,-1.66,-0.63,NaN,NaN,0.77,NaN,NaN,NaN,0.79,NaN,NaN,NaN,NaN,NaN,NaN,-0.25,NaN,NaN,0.39,2.82,3.52,NaN,0.72,0.62,-1.01,-0.97,0.02,NaN,0.02,NaN,1.47,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.26,-2.20,0.66,-2.20,NaN,0.03,0.01,0.30,-0.14,-1.95,NaN,0.99,-0.08
1,2,2.55,-0.93,NaN,0.17,0.27,0.08,NaN,-0.40,-0.54,-1.87,1.22,NaN,NaN,0.08,0.85,-0.76,-0.02,2.01,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,-0.10,0.49,0.00,1.24,3.02,-0.26,NaN,NaN,-2.11,NaN,NaN,0.66,0.96,NaN,NaN,NaN,NaN,NaN,NaN,2.19,NaN,NaN,-1.22,-0.22,-3.60,NaN,0.91,-0.97,-0.77,-0.97,0.02,NaN,0.01,-0.09,-3.61,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.24,-0.80,0.57,-0.81,NaN,0.16,0.00,0.19,1.18,0.12,NaN,-0.22,0.06
2,3,-3.73,0.99,NaN,0.15,0.16,0.40,NaN,-0.66,1.51,-1.66,-0.90,NaN,NaN,1.00,0.59,-3.18,-0.71,-2.74,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,-0.09,0.45,0.00,1.43,1.01,-0.26,NaN,NaN,0.99,NaN,NaN,0.67,0.46,NaN,NaN,NaN,NaN,NaN,NaN,-0.13,NaN,NaN,0.44,-0.55,3.35,NaN,1.52,0.68,-0.85,0.37,0.02,NaN,0.11,0.22,0.45,NaN,0.00,-1.14,NaN,-4.00,NaN,NaN,NaN,0.76,NaN,0.22,0.15,0.52,0.21,NaN,0.04,0.00,0.18,-0.73,-0.37,NaN,0.36,0.00
3,4,2.12,0.04,NaN,0.12,0.76,-0.02,NaN,-0.27,-1.61,-1.77,-1.16,NaN,NaN,-0.22,0.43,-3.99,0.73,0.27,3.09,0.93,-0.98,-0.04,NaN,NaN,NaN,NaN,NaN,-0.09,NaN,0.09,0.35,0.44,0.00,1.53,1.90,0.24,NaN,NaN,0.10,NaN,NaN,0.81,0.65,NaN,NaN,NaN,NaN,NaN,NaN,-0.49,NaN,NaN,3.45,0.87,3.03,NaN,1.41,0.02,2.46,-0.97,0.02,-1.61,1.00,-0.05,-0.91,-0.19,-0.28,-2.55,NaN,-4.00,NaN,NaN,NaN,0.55,NaN,0.19,-0.37,0.49,-0.40,NaN,0.08,0.00,0.33,-0.40,-0.19,NaN,0.01,-0.03
4,5,-0.85,-0.02,NaN,0.22,0.12,-0.27,NaN,1.56,-0.36,-2.62,-1.59,NaN,1.15,0.52,0.32,-2.02,0.88,-0.81,-0.07,1.71,0.42,-0.83,NaN,NaN,NaN,NaN,NaN,0.01,NaN,0.09,0.21,0.43,0.00,1.49,1.88,NaN,NaN,NaN,0.70,NaN,NaN,0.87,2.47,NaN,NaN,NaN,NaN,NaN,NaN,-0.65,NaN,NaN,1.20,2.58,3.76,NaN,0.93,0.90,-1.49,0.40,0.02,NaN,0.03,-0.04,-0.94,-0.19,-0.05,-3.11,NaN,0.06,NaN,NaN,NaN,0.34,NaN,0.06,1.64,0.23,1.50,NaN,0.05,0.00,0.54,0.51,-0.08,NaN,-0.01,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625,1.14,1.10,1.11,0.07,0.00,0.50,-3.97,-0.10,-0.14,-3.05,-3.51,-2.07,NaN,NaN,NaN,-2.19,-0.19,1.33,-1.53,NaN,NaN,NaN,1.37,1.12,-1.67,0.21,-0.47,-0.34,NaN,0.05,0.43,0.86,1.00,0.34,3.12,NaN,-1.41,NaN,-1.44,-1.19,NaN,NaN,0.34,0.56,0.25,NaN,NaN,NaN,0.18,-1.02,0.31,2.77,-4.00,-2.42,3.92,NaN,-0.87,0.24,-0.04,-0.97,0.24,NaN,NaN,NaN,-4.00,NaN,0.01,0.37,NaN,0.00,NaN,0.50,-1.33,0.42,0.28,0.08,0.13,0.19,-0.80,NaN,0.14,0.02,0.46,NaN,0.10,3.48,-0.28,0.00
105438,173659,-1.37,0.29,NaN,0.06,0.00,0.16,NaN,0.34,0.24,-2.28,-1.51,-0.46,NaN,NaN,NaN,NaN,0.09,0.69,0.04,NaN,NaN,NaN,-2.43,1.14,-4.00,2.44,-0.18,NaN,NaN,0.25,0.24,NaN,0.00,0.00,2.78,-0.04,NaN,NaN,NaN,0.52,NaN,NaN,0.55,-1.46,-0.62,NaN,NaN,NaN,0.34,1.89,0.39,-0.12,1.31,-0.05,NaN,NaN,NaN,-0.78,NaN,-0.78,-0.24,NaN,NaN,NaN,2.78,NaN,-0.03,0.50,NaN,0.00,NaN,0.12,1.80,0.22,0.06,NaN,NaN,0.19,-1.41,NaN,NaN,NaN,NaN,NaN,0.92,1.09,0.98,-0.01
105439,173670,0.05,0.34,NaN,0.08,0.22,-0.16,NaN,0.18,0.18,-1.16,-2.15,-1.16,NaN,NaN,NaN,-2.17,-0.42,1.29,-1.02,NaN,NaN,NaN,-3.83,-0.30,-2.54,0.84,-0.37,-0.53,NaN,0.19,0.28,-1.21,0.00,0.34,3.12,-0.01,-2.00,NaN,NaN,0.47,NaN,NaN,0.42,-1.32,-0.38,-0.50

In [10]:
# df = df.fillna(0)

In [11]:
df['EMA'] = df['target'].ewm(span=10, adjust=False).mean()

In [12]:
df

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,target,EMA
0,1,1.65,-3.48,NaN,0.12,0.22,0.20,NaN,-0.29,-0.36,-1.88,-2.82,NaN,NaN,NaN,0.09,-0.72,0.12,1.11,3.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.41,-0.41,0.44,0.00,NaN,-1.66,-0.63,NaN,NaN,0.77,NaN,NaN,NaN,0.79,NaN,NaN,NaN,NaN,NaN,NaN,-0.25,NaN,NaN,0.39,2.82,3.52,NaN,0.72,0.62,-1.01,-0.97,0.02,NaN,0.02,NaN,1.47,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.26,-2.20,0.66,-2.20,NaN,0.03,0.01,0.30,-0.14,-1.95,NaN,0.99,-0.08,-0.08
1,2,2.55,-0.93,NaN,0.17,0.27,0.08,NaN,-0.40,-0.54,-1.87,1.22,NaN,NaN,0.08,0.85,-0.76,-0.02,2.01,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,-0.10,0.49,0.00,1.24,3.02,-0.26,NaN,NaN,-2.11,NaN,NaN,0.66,0.96,NaN,NaN,NaN,NaN,NaN,NaN,2.19,NaN,NaN,-1.22,-0.22,-3.60,NaN,0.91,-0.97,-0.77,-0.97,0.02,NaN,0.01,-0.09,-3.61,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.24,-0.80,0.57,-0.81,NaN,0.16,0.00,0.19,1.18,0.12,NaN,-0.22,0.06,-0.05
2,3,-3.73,0.99,NaN,0.15,0.16,0.40,NaN,-0.66,1.51,-1.66,-0.90,NaN,NaN,1.00,0.59,-3.18,-0.71,-2.74,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,-0.09,0.45,0.00,1.43,1.01,-0.26,NaN,NaN,0.99,NaN,NaN,0.67,0.46,NaN,NaN,NaN,NaN,NaN,NaN,-0.13,NaN,NaN,0.44,-0.55,3.35,NaN,1.52,0.68,-0.85,0.37,0.02,NaN,0.11,0.22,0.45,NaN,0.00,-1.14,NaN,-4.00,NaN,NaN,NaN,0.76,NaN,0.22,0.15,0.52,0.21,NaN,0.04,0.00,0.18,-0.73,-0.37,NaN,0.36,0.00,-0.04
3,4,2.12,0.04,NaN,0.12,0.76,-0.02,NaN,-0.27,-1.61,-1.77,-1.16,NaN,NaN,-0.22,0.43,-3.99,0.73,0.27,3.09,0.93,-0.98,-0.04,NaN,NaN,NaN,NaN,NaN,-0.09,NaN,0.09,0.35,0.44,0.00,1.53,1.90,0.24,NaN,NaN,0.10,NaN,NaN,0.81,0.65,NaN,NaN,NaN,NaN,NaN,NaN,-0.49,NaN,NaN,3.45,0.87,3.03,NaN,1.41,0.02,2.46,-0.97,0.02,-1.61,1.00,-0.05,-0.91,-0.19,-0.28,-2.55,NaN,-4.00,NaN,NaN,NaN,0.55,NaN,0.19,-0.37,0.49,-0.40,NaN,0.08,0.00,0.33,-0.40,-0.19,NaN,0.01,-0.03,-0.04
4,5,-0.85,-0.02,NaN,0.22,0.12,-0.27,NaN,1.56,-0.36,-2.62,-1.59,NaN,1.15,0.52,0.32,-2.02,0.88,-0.81,-0.07,1.71,0.42,-0.83,NaN,NaN,NaN,NaN,NaN,0.01,NaN,0.09,0.21,0.43,0.00,1.49,1.88,NaN,NaN,NaN,0.70,NaN,NaN,0.87,2.47,NaN,NaN,NaN,NaN,NaN,NaN,-0.65,NaN,NaN,1.20,2.58,3.76,NaN,0.93,0.90,-1.49,0.40,0.02,NaN,0.03,-0.04,-0.94,-0.19,-0.05,-3.11,NaN,0.06,NaN,NaN,NaN,0.34,NaN,0.06,1.64,0.23,1.50,NaN,0.05,0.00,0.54,0.51,-0.08,NaN,-0.01,0.06,-0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625,1.14,1.10,1.11,0.07,0.00,0.50,-3.97,-0.10,-0.14,-3.05,-3.51,-2.07,NaN,NaN,NaN,-2.19,-0.19,1.33,-1.53,NaN,NaN,NaN,1.37,1.12,-1.67,0.21,-0.47,-0.34,NaN,0.05,0.43,0.86,1.00,0.34,3.12,NaN,-1.41,NaN,-1.44,-1.19,NaN,NaN,0.34,0.56,0.25,NaN,NaN,NaN,0.18,-1.02,0.31,2.77,-4.00,-2.42,3.92,NaN,-0.87,0.24,-0.04,-0.97,0.24,NaN,NaN,NaN,-4.00,NaN,0.01,0.37,NaN,0.00,NaN,0.50,-1.33,0.42,0.28,0.08,0.13,0.19,-0.80,NaN,0.14,0.02,0.46,NaN,0.10,3.48,-0.28,0.00,0.00
105438,173659,-1.37,0.29,NaN,0.06,0.00,0.16,NaN,0.34,0.24,-2.28,-1.51,-0.46,NaN,NaN,NaN,NaN,0.09,0.69,0.04,NaN,NaN,NaN,-2.43,1.14,-4.00,2.44,-0.18,NaN,NaN,0.25,0.24,NaN,0.00,0.00,2.78,-0.04,NaN,NaN,NaN,0.52,NaN,NaN,0.55,-1.46,-0.62,NaN,NaN,NaN,0.34,1.89,0.39,-0.12,1.31,-0.05,NaN,NaN,NaN,-0.78,NaN,-0.78,-0.24,NaN,NaN,NaN,2.78,NaN,-0.03,0.50,NaN,0.00,NaN,0.12,1.80,0.22,0.06,NaN,NaN,0.19,-1.41,NaN,NaN,NaN,NaN,NaN,0.92,1.09,0.98,-0.01,0.00
105439,173670,0.05,0.34,NaN,0.08,0.22,-0.16,NaN,0.18,0.18,-1.16,-2.15,-1.16,NaN,NaN,NaN,-2.17,-0.42,1.29,-1.02,NaN,NaN,NaN,-3.83,-0.30,-2.54,0.84,-0.37,-0.53,NaN,0.19,0.28,-1.21,0.00,0.34,3.12,-0.01,-

In [13]:
X = df.drop(columns=['row_id', 'target'])
y = df['target']

In [14]:
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Initialize the XGBRegressor
model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=1, alpha=10, n_estimators=2000)

# Train the model with the selected features
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
print("mse:", mse)

mse: 0.002517829073798468


In [16]:
df_test = pd.read_csv('/kaggle/input/trexquant-2025-summer-intern-challenge-iitkgp/TQ_Test_Kaggle.csv')
df_test

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87
0,44,-0.87,0.39,1.80,0.18,0.03,0.11,0.22,0.03,-1.02,-2.49,-1.87,-0.44,-0.07,0.85,1.18,-1.04,0.24,1.12,1.10,NaN,NaN,-0.40,-0.70,-0.02,1.01,-0.63,-0.44,0.08,0.01,0.11,0.00,0.83,2.00,1.61,2.78,-0.03,-1.33,-0.83,-0.05,0.18,-0.67,-0.89,-1.95,0.06,0.02,0.66,0.47,-2.36,0.06,1.74,0.26,-0.22,0.34,-0.45,3.52,0.00,1.07,0.97,-1.90,0.27,0.62,NaN,0.00,-0.03,-1.81,-0.05,0.08,-0.40,0.26,-0.13,0.18,0.22,-2.27,0.47,0.39,0.13,-2.02,0.25,-2.01,-0.70,0.11,0.00,0.12,1.17,-0.96,3.70,0.05
1,45,-1.07,-0.00,2.83,0.10,0.07,0.26,NaN,NaN,0.71,-1.41,-1.72,0.54,0.49,0.77,-0.12,-0.49,0.77,1.12,-0.11,NaN,NaN,-1.01,0.27,0.66,-1.44,-0.95,-0.40,-0.12,-0.15,0.10,0.01,0.91,0.00,1.55,3.30,-0.03,-2.00,0.63,-0.03,-0.76,-1.69,-0.89,-0.89,-0.47,0.06,-2.31,-0.98,-2.44,0.06,0.96,0.29,0.01,1.07,0.89,-1.82,NaN,0.34,0.87,1.41,1.00,0.33,NaN,0.01,0.00,0.91,0.14,0.00,-0.54,0.18,-0.06,0.50,0.41,-2.46,0.48,0.64,0.12,-1.65,0.28,-1.65,1.38,0.12,0.00,0.13,0.87,-0.70,3.72,0.20
2,46,-0.61,0.99,-2.31,0.16,0.40,0.05,-3.01,-1.50,0.36,-2.20,0.16,0.39,0.42,0.25,-0.64,-0.58,-0.54,1.21,1.96,NaN,1.37,1.72,0.62,0.42,-0.97,0.37,-0.49,-0.12,-0.27,0.09,0.02,0.86,1.00,1.65,3.11,-0.05,-2.93,-1.71,-0.85,-1.51,0.13,-0.89,-1.19,-0.14,-0.07,-3.61,-3.59,-2.17,0.13,2.11,0.13,0.86,0.00,1.92,-0.28,NaN,0.85,0.33,-1.98,-0.87,0.17,NaN,0.03,0.06,-2.62,0.12,0.06,-0.34,0.25,0.23,1.00,0.19,-1.71,0.52,0.36,0.15,1.20,0.31,1.11,-0.57,0.12,0.00,0.09,1.03,-0.53,3.79,0.11
3,47,2.63,-0.34,0.48,0.15,0.10,-0.02,NaN,1.71,0.77,-1.29,-0.95,0.80,0.44,1.28,0.25,-0.55,-0.05,1.48,1.34,NaN,0.41,1.06,1.63,0.22,-0.27,1.84,0.32,-0.20,-0.06,0.09,0.02,0.82,2.00,1.74,0.52,-0.05,-2.64,NaN,0.64,-0.23,-0.01,-0.89,1.37,0.05,0.06,1.78,1.23,-1.73,0.24,0.20,0.86,0.57,2.80,3.11,-3.19,NaN,1.97,1.00,0.77,1.00,0.62,NaN,0.59,0.08,2.35,0.09,-0.05,-1.03,0.39,0.00,0.86,0.29,-2.36,0.31,0.43,0.12,-0.47,0.23,-0.48,2.23,0.11,0.00,0.10,1.97,-0.51,3.64,0.08
4,48,0.66,-0.85,-0.98,0.18,1.15,0.48,NaN,0.50,-0.62,-0.58,-2.57,-1.06,NaN,0.30,0.63,-0.97,0.00,1.05,-0.53,NaN,-0.34,-0.43,-0.75,-0.17,0.64,0.80,0.24,-0.11,0.43,0.08,0.03,0.82,0.00,0.95,1.94,-0.05,-2.40,NaN,-0.36,-0.66,0.80,0.13,-1.47,0.91,0.12,-0.03,-0.03,-1.24,0.39,1.63,0.24,0.15,-2.50,-1.39,3.03,NaN,1.50,0.62,1.33,-0.84,-0.21,NaN,NaN,0.02,0.02,0.07,0.05,-0.25,0.24,-0.41,2.52,0.33,-2.27,0.35,0.32,0.13,-1.31,0.26,-1.33,NaN,0.39,0.00,0.13,2.22,-0.68,3.72,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85816,191259,-0.37,-0.35,-0.40,0.09,0.65,0.47,NaN,0.00,-0.32,-2.56,-2.19,-0.55,NaN,NaN,-0.22,-3.58,-0.16,0.88,1.93,NaN,NaN,0.58,-0.45,0.62,-1.36,-0.13,-0.49,-0.16,0.27,0.32,-0.11,-1.10,1.00,0.04,2.88,-0.02,-1.33,-0.71,-3.46,0.53,1.29,-0.12,0.42,-0.09,-0.12,NaN,NaN,-1.87,0.09,2.94,0.30,0.61,1.11,-1.13,-3.27,NaN,1.10,0.52,2.14,-0.46,-0.02,NaN,0.00,NaN,1.13,NaN,0.11,0.22,NaN,-0.03,NaN,0.12,-0.79,0.26,0.49,0.07,0.71,0.15,0.45,NaN,0.02,0.00,0.78,0.49,2.27,3.49,0.55
85817,191260,0.75,1.22,NaN,0.02,0.14,-0.21,NaN,1.58,-0.04,-2.82,-1.76,-1.96,NaN,NaN,NaN,-1.22,-0.82,0.44,4.00,NaN,NaN,NaN,0.20,0.14,-0.80,1.25,0.44,-0.06,0.31,0.11,0.10,-1.00,1.00,0.05,1.38,-0.02,-2.00,-0.70,1.71,NaN,NaN,-0.56,-0.57,-0.50,-0.03,NaN,NaN,-2.20,0.11,1.97,0.95,-0.61,-0.70,1.64,-0.36,NaN,-1.83,-0.97,-0.28,0.94,0.14,2.04,0.03,NaN,4.00,NaN,-0.19,0.56,NaN,-0.07,NaN,0.16,-1.58,0.30,0.37,0.06,-0.15,0.14,-0.10,-0.89,0.13,0.00,0.48,NaN,2.11,2.94,-0.10
85818,191261,-0.04,0.39,-0.86,0.05,0.07,-0.03,N

In [18]:
X_test = df_test.drop(columns=['row_id'])

In [21]:
for _, row in X_test.iterrows():
    print(row[0])
    break

-0.8693381


/tmp/ipykernel_34/3033448371.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])


In [22]:
y_test = []

for i, row in X_test.iterrows():
    temp_df = row.to_frame().T
    
    if y_test:
        temp_df['EMA'] = pd.Series(y_test).ewm(span=10, adjust=False).mean().iloc[-1]
    else:
        temp_df['EMA'] = 0  # Set EMA to 0 for the first prediction
    
    temp_df = temp_df[X_test.columns.tolist() + ['EMA']]    
    pred = model.predict(temp_df)
    y_test.append(pred[0])


In [23]:
# Create DataFrame for predictions
df_output = pd.DataFrame({"row_id": np.array(df_test['row_id']), 'target': np.array(y_test)})

# Define filename for saving predictions
filename = "/kaggle/working/predictions_xgb_past_10_0_impute_2k_estimator.csv"

# Save predictions to CSV file
df_output.to_csv(filename, index=False)

print(f"Successfully Saved in File: {filename}")

Successfully Saved in File: /kaggle/working/predictions_xgb_past_10_0_impute_2k_estimator.csv


In [24]:
df_output

,row_id,target
0,44,-0.00
1,45,-0.01
2,46,-0.02
3,47,-0.04
4,48,-0.04
...,...,...
85816,191259,-0.14
85817,191260,-0.16
85818,191261,-0.14
85819,191262,-0.14
